# Microsoft L2R Datasets
[MS datasets](https://www.microsoft.com/en-us/research/project/mslr/)

`MSLR-WEB30k`と`MSLR-WEB10K`といった二つの大きなデータセットを公開している。

## Datasets Desc
+ クエリとURLはIDにより表現されている
+ 特徴量と関連度の程度のペアで表現されている

(1) 関連度はすでに利用されなくなった、商用Web検索エンジンのラベリングを利用している。
関連度は0~4のラベルで表現されている。

(2) 素性はMicrosoftにより設定されているが、大抵の検索エンジンで利用されているもの。

関連度と素性の組み、クエリIDは次のように表現されている。

```dat
0 qid:1 1:3 2:0 3:2 4:2 … 135:0 136:0
2 qid:1 1:3 2:3 3:0 4:0 … 135:0 136:0
...
```

## Feature List
`query-url`ペアは136次元のベクトルにより表現されている。大別したものを表に表す。

| feature name | desc |
|:-----------------|:-------|
| covered query term number | 文書に含まれるクエリに利用された単語数 |
| covered query term ratio | 文書に含まれるクエリに利用された単語数 / クエリの単語数 |
| stream length | ??? |
| IDF | 逆文書頻度 クエリの単語を含む文書数の逆数だったはず |
| sum of term freq | クエリの単語をいくつ含むか　|
| min of term freq | クエリに含まれる単語の中で、文書中で最も出現しなかった単語の出現数 |
| max of term freq | 上の逆バージョン　|
| mean of term freq | 上の平均バージョン　|
| variance of term freq | 上の分散バージョン |
| sum of tf\* idf | クエリの単語のTF-IDFの総和 |
| boolean model | 文書とクエリの論理値モデルのスコア |
| vector space model | 文書とクエリのベクトル空間モデルのスコア |
| BM25 | okapiのやつ？ |

端的に言ってしまえば、いろいろなスコアを文書について設定して、それぞれのスコアから`Ground-truth`と最も矛盾が少なく済むように素性に対する重みベクトルを決定することを目標としている。

## データ利用
ダウンロードできるデータの形式が特殊なので、CSV形式に変換するスクリプトを考える。

```
0 qid:1 1:3 2:0 3:2 4:2 … 135:0 136:0
```

一行あたりの表現。半角スペースで分割されている。
最初の数値は関連度、2番目はクエリID、それ以降はそれぞれの素性の`次元:値`となっている。

In [1]:
import pandas as pd
from numba import jit
import time

In [2]:
def read_dat(filepath="./datasets/Fold1/train.txt", show_proc=True):
    """DataFrameに読み込ませるリストを返す"""
    rows = []
    _append = rows.append # 高速化
    with  open(filepath, "r") as f:
        lines = f.readlines()
    for i, line in enumerate(lines):
        if show_proc and i % 100000 == 0:
            print("{0}~{1} lines are being precessed...".format(i, i + 99999))
        li = line.split()
        li_dict = { "rel": li[0] }
        for idx in range(1, len(li)):
            kv = li[idx].split(":")
            li_dict[kv[0]] = kv[1] # 型変換はpandasで
        _append(li_dict)
    return rows

In [3]:
# パフォーマンスの計測
def fib(n):
    if n == 0: return 1
    if n == 1: return 1
    if n >= 2: return fib(n-1) + fib(n-2)

start = time.time()
fib(10)
time_elapsed = time.time() - start
print("time_slapsed: {0} sec".format(time_elapsed))

time_slapsed: 7.605552673339844e-05 sec


In [4]:
# 読み込み速度の計測
# s1 = time.time()
# rows_ = naive_read_dat()
# t1 = time.time() - s1
# print("time_slapsed: {0} sec".format(t1))

s2 = time.time()
rows = read_dat()
t2 = time.time() - s2
print("time_slapsed: {0} sec".format(t2))

0~99999 lines are being precessed...
100000~199999 lines are being precessed...
200000~299999 lines are being precessed...
300000~399999 lines are being precessed...
400000~499999 lines are being precessed...
500000~599999 lines are being precessed...
600000~699999 lines are being precessed...
700000~799999 lines are being precessed...
time_slapsed: 62.31073808670044 sec


##  方針
学習器の出力はdf["rel"]の予測値。その予測値は0~4を離散的に取りうる。
同じクエリに関連するドキュメント対を比較し、`pairwise preference`を関連度から取得する。

### RankingSVM
訓練データ$D = \{(\hat{d_{i}}, \check{d_{j}})\}_{i=1}^{N}$は、クエリに関連するドキュメントの対$(\hat{d_{i}}, \check{d_{j}})$を$N$件持っている。ただし、$\hat{d_{i}}$は$\check{d_{j}}$よりも関連度が高い文書である。
それぞれの文書がベクトル$\hat{x_{i}}$、$\check{x_{j}}$で表される時、RankingSVMは次のような最適化問題を考える。

$$
min: L(w, \xi) = \frac{1}{2}w^{T}w + C\sum_{i}^{N}\xi_{i}
$$
$$
s.t. {w}^{T}\phi(\hat{x_{i}}) \ge w^{T}\phi(\check{x_{j}}) + 1 - \xi_{i}
$$

学習された重み$\bf{w^{*}}$を利用して、ランキング関数は次のように表現される。

$$
f(d) = w^{*T}\phi(x)
$$

### 実装
次のページを参考にした。

[ranking svm](https://gist.github.com/fabianp/2020955)

In [5]:
df = pd.DataFrame(rows)
df.head()

,1,10,100,101,102,103,104,105,106,107,...,92,93,94,95,96,97,98,99,qid,rel
0,3,1,1,1,1,0,0,1,12.941469,20.59276,...,1.084169,0,0,2.78795,1,1,0,0,1,2
1,3,1,1,0.994425,0,1,0,0.995455,20.885118,0,...,0,1.167248,0,75.339929,1,0,1,0,1,2
2,3,1,1,0.851903,0,0.720414,0,0.842789,18.140878,0,...,0,7.736505,0,61.124916,1,0,0,0,1,0
3,3,1,1,0.989585,0,1,0,0.995185,15.572998,0,...,0,1.167248,0,3.900733,1,0,1,0,1,2
4,3,1,1,0.980551,0,1,0,0.989938,7.802556,0,...,0,1.167248,0,6.941327,1,0,1,0,1,1


In [6]:
import itertools
import numpy as np
from sklearn import svm, linear_model, cross_validation

def transform_pairwise(X, y):
    """文書集合の関連度をみて、pairwise preferenceに変換する
    
    注意するべきは、Gistの実装とは違い、複数のクエリを考える必要があること
    """
    X_new = []
    y_new = []
    y = np.asarray(y)
    if y.ndim == 1:
        y = np.c_[y, np.ones(y.shape[0])]
    comb = itertools.combinations(range(X.shape[0]), 2)
    for k, (i, j) in enumerate(comb):
        if y[i, 0] == y[j, 0] or y[i, 1] != y[j, 1]:
            continue
        X_new.append(X[i] - X[j])
        y_new.append(np.sign(y[i, 0] - y[j, 0]))
        # y_newの符号に偏りが出ないようにする
        if y_new[-1] != (-1) ** k:
            y_new[-1] = - y_new[-1]
            X_new[-1] = - X_new[-1]
    return np.asarray(X_new), np.asarray(y_new).ravel()

/Users/SaigusaMoriaki/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [14]:
grouped = df.groupby("qid")
cols = df.columns
X_feat = list(cols[:-2]) # -2, -1番目はqidとrel
y_feat = cols[-1] # rel
# より小さなデータセットで試す n_iter = 10
# data_X, data_y
# g は[0]がグループのインデックス, [1]がpd.DataFrame
X_train = np.array([])
y_train = np.array([])
X_test = np.array([])
y_test = np.array([])
for _, g in enumerate(grouped):
    tmp = g[1]
    if _ <= 10:
        X = tmp[X_feat].applymap(float).values
        y = tmp[y_feat].map(float)
        X_new, y_new = transform_pairwise(X, y)
        if _ <= 9:
            X_train = np.append(X_train, X_new, axis=0)
            y_train = np.append(y_train, y_new)
        else:
            X_test = np.append(X_test, X_new, axis=0)
            y_test = np.append(y_test, y_new)

In [16]:
# clf = svm.SVC()
# clf.fit(X_new, y_new)

array([-1., -1.,  1., ...,  1., -1., -1.])

次はクエリごとにデータセットを作成するのと、テストデータについてうまく学習できているかを確認するところから